## Imports

In [1]:
import requests
import pandas as pd
import re
import numpy as np
import random
import glob
import os
import googlemaps
import folium

from bs4 import BeautifulSoup
from IPython.display import clear_output

from itertools import cycle
import urllib.request
from urllib3 import ProxyManager, make_headers

from tqdm import tqdm
from time import sleep

## Color-code top sale

In [43]:
### Insert RANK values ###
df_final_top_ten['RANK'] = range(1, len(df_final_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_final_top_ten['COLOR'] = np.where(df_final_top_ten['RANK'] <= 1, 'orange', 'blue')

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_32338/2673149807.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_top_ten['RANK'] = range(1, len(df_final_top_ten) + 1)
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_32338/2673149807.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_top_ten['COLOR'] = np.where(df_final_top_ten['RANK'] <= 1, 'orange', 'blue')


In [44]:
df_final_top_ten.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'LOCATION', 'SQUARE FEET',
       'DAYS ON MARKET', '$/SQUARE FEET', 'LATITUDE', 'LONGITUDE', 'LATITUDE',
       'LONGITUDE', 'LIST AGENTS', 'LIST COMPANIES', 'BOUGHT AGENTS',
       'BOUGHT COMPANIES', 'RANK', 'COLOR'],
      dtype='object')

In [45]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    listing_agent = row['LIST AGENTS']
    buyers_agent = row['BOUGHT AGENTS']
    psf = row['$/SQUARE FEET']
    address = row['ADDRESS']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [52]:
df_final_top_ten = df_final_top_ten.loc[:,~df_final_top_ten.columns.duplicated()].copy()

In [56]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df_final_top_ten[["LATITUDE", "LONGITUDE"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('July 30th - August 7th')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_final_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [58]:
m.save('index.html')

## Data Snagger

In [59]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [61]:
df_final_top_ten.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'LOCATION', 'SQUARE FEET',
       'DAYS ON MARKET', '$/SQUARE FEET', 'LATITUDE', 'LONGITUDE',
       'LIST AGENTS', 'LIST COMPANIES', 'BOUGHT AGENTS', 'BOUGHT COMPANIES',
       'RANK', 'COLOR'],
      dtype='object')

In [84]:
df_final_top_ten.iloc[1]

SOLD DATE                   July-31-2023
ADDRESS             19575 Collins Ave #6
CITY                   Sunny Isles Beach
PRICE                            7750000
LOCATION                   REGALIA CONDO
SQUARE FEET                       4992.0
DAYS ON MARKET                       NaN
$/SQUARE FEET                     1552.0
LATITUDE                       25.956967
LONGITUDE                     -80.119435
LIST AGENTS                 Chad Carroll
LIST COMPANIES      Compass Florida, LLC
BOUGHT AGENTS               Chad Carroll
BOUGHT COMPANIES    Compass Florida, LLC
RANK                                   2
COLOR                               blue
Name: 1, dtype: object

In [63]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmax()]['LOCATION']}, {df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Listing agent: {df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmax()]['LIST AGENTS']} with {df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmax()]['LIST COMPANIES']} | Buyer's agent: {df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmax()]['BOUGHT AGENTS']} with {df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmax()]['BOUGHT AGENTS']}")
print(f"{LE}{BR}{df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmin()]['LOCATION']}, {df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmin()]['ADDRESS']} | Price ${df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Listing agent: {df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmin()]['LIST AGENTS']} with {df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmin()]['LIST COMPANIES']} | Buyer's agent: {df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmin()]['BOUGHT AGENTS']} with {df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmin()]['BOUGHT AGENTS']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmax()]['LOCATION']}, {df_final_top_ten.loc[df_final_top_ten['$/SQUARE FEET'].idxmax()]['ADDRESS']} | Price ${df_final_top_ten.loc[df_final_top_ten['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_final_top_ten.loc[df_final_top_ten['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Listing agent: {df_final_top_ten.loc[df_final_top_ten['$/SQUARE FEET'].idxmax()]['LIST AGENTS']} with {df_final_top_ten.loc[df_final_top_ten['$/SQUARE FEET'].idxmax()]['LIST COMPANIES']} | Buyer's agent: {df_final_top_ten.loc[df_final_top_ten['$/SQUARE FEET'].idxmax()]['BOUGHT AGENTS']} with {df_final_top_ten.loc[df_final_top_ten['$/SQUARE FEET'].idxmax()]['BOUGHT COMPANIES']}")
print(f"{MIN_PSF}{BR}{df_final_top_ten.loc[df_final_top_ten['PRICE'].idxmin()]['LOCATION']}, {df_final_top_ten.loc[df_final_top_ten['$/SQUARE FEET'].idxmin()]['ADDRESS']} | Price ${df_final_top_ten.loc[df_final_top_ten['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_final_top_ten.loc[df_final_top_ten['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Listing agent: {df_final_top_ten.loc[df_final_top_ten['$/SQUARE FEET'].idxmin()]['LIST AGENTS']} with {df_final_top_ten.loc[df_final_top_ten['$/SQUARE FEET'].idxmin()]['LIST COMPANIES']} | Buyer's agent: {df_final_top_ten.loc[df_final_top_ten['$/SQUARE FEET'].idxmin()]['BOUGHT AGENTS']} with {df_final_top_ten.loc[df_final_top_ten['$/SQUARE FEET'].idxmin()]['BOUGHT COMPANIES']}")

Most Expensive
SURF CLUB CONDO, 9001 Collins Ave Unit S-207 | Price $18,500,000 | $4,229 psf | Listing agent:  Ximena Penuela,  Kimberlyn Montilva with Fort Realty, LLC, Fort Realty, LLC | Buyer's agent:  Dustin Nero with  Dustin Nero
Least Expensive
BLAIR HOUSE CONDO - WEST, 9100 W Bay Harbor Dr Unit 10D | Price $1,900,000 | $640 psf | Listing agent:  Cory Waldman with Coldwell Banker Realty | Buyer's agent:  Cory Waldman with  Cory Waldman
Highest Price Per Square Foot
SURF CLUB CONDO, 9001 Collins Ave Unit S-207 | Price $18,500,000 | $4,229 psf | Listing agent:  Ximena Penuela,  Kimberlyn Montilva with Fort Realty, LLC, Fort Realty, LLC | Buyer's agent:  Dustin Nero with Douglas Elliman
Lowest Price Per Square Foot
BLAIR HOUSE CONDO - WEST, 9100 W Bay Harbor Dr Unit 10D | Price $1,900,000 | $640 psf | Listing agent:  Cory Waldman with Coldwell Banker Realty | Buyer's agent:  Cory Waldman with Coldwell Banker Realty


## All data summary

In [71]:
print(len(df))

193


In [77]:
print(df['PRICE'].max())

18500000


In [74]:
print(df['$/SQUARE FEET'].mean())

537.4368421052632


## Time on market

In [85]:
from datetime import datetime, timedelta

date1 = datetime(2023, 1, 10) ## List (Earlier) date
date2 = datetime(2023, 7, 31) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

202


## Map URL Snagger

In [2]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [3]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_weekly_condo_sales_no_tabula
